### Setup and Connect to Neo4j Aura


In [12]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [13]:
# Update these with your Neo4j Aura credentials
AURA_URI = os.getenv('NEO4J_URL')
AURA_USERNAME = os.getenv('NEO4J_USERNAME')
AURA_PASSWORD = os.getenv('NEO4J_PASSWORD')

from neo4j import GraphDatabase

# Connect to Neo4j Aura
driver = GraphDatabase.driver(AURA_URI, auth=(AURA_USERNAME, AURA_PASSWORD))
print("Connected to Neo4j Aura!")

Connected to Neo4j Aura!


### Clear the database

In [14]:
# Define function to clear the database
def clear_database(tx):
    tx.run("MATCH (n) DETACH DELETE n")
    print("Database cleared.")
with driver.session() as session:
    session.execute_write(clear_database)

Database cleared.


### Add a Node with Properties

In [15]:
def add_person(tx, name: str, age: int, city: str):
    tx.run("CREATE (p:Person {name: $name, age: $age, city: $city})", 
           name=name, age=age, city=city)
    print(f"Added Person: {name}, Age: {age}, City: {city}")

with driver.session() as session:
    session.execute_write(add_person, name="Alice", age=25, city="New York")

Added Person: Alice, Age: 25, City: New York


### Add Another Node with Properties

In [16]:
def add_person(tx, name: str, age: int, city: str):
    tx.run("CREATE (p:Person {name: $name, age: $age, city: $city})", 
           name=name, age=age, city=city)
    print(f"Added Person: {name}, Age: {age}, City: {city}")

with driver.session() as session:
    session.execute_write(add_person, name="Bob", age=30, city="London")

Added Person: Bob, Age: 30, City: London


### Add a Relationship

In [17]:
def add_relationship(tx, name1: str, name2: str, rel_type: str):
    tx.run(
        "MATCH (p1:Person {name: $name1}), (p2:Person {name: $name2}) "
        "CREATE (p1)-[r:" + rel_type + "]->(p2)",
        name1=name1, name2=name2
    )
    print(f"Added {rel_type} relationship: {name1} -> {name2}")

with driver.session() as session:
    session.execute_write(add_relationship, name1="Alice", name2="Bob", rel_type = "FRIEND_WITH")

Added FRIEND_WITH relationship: Alice -> Bob


In [18]:
def remove_relationship(tx, name1: str, name2: str, rel_type: str):
    tx.run(
        "MATCH (p1:Person {name: $name1})-[r:" + rel_type + "]->(p2:Person {name: $name2}) "
        "DELETE r",
        name1=name1, name2=name2
    )
    print(f"Removed {rel_type} relationship: {name1} -> {name2}")
    
with driver.session() as session:
    session.execute_write(remove_relationship, name1="Alice", name2="Bob", rel_type = "BEST_FRIENDS_WITH")

Removed BEST_FRIENDS_WITH relationship: Alice -> Bob


### Query a Node's Properties

In [19]:
def get_person_details(tx, name: str):
    result = tx.run("MATCH (p:Person {name: $name}) RETURN p.name, p.age, p.city", name=name)
    for record in result:
        print(f"Name: {record['p.name']}, Age: {record['p.age']}, City: {record['p.city']}")

with driver.session() as session:
    session.execute_read(get_person_details, name="Alice")

Name: Alice, Age: 25, City: New York


### Query Relationships

In [20]:
def get_friends(tx, name: str):
    result = tx.run(
        "MATCH (p:Person {name: $name})-[:FRIENDS_WITH]->(friend:Person) "
        "RETURN friend.name, friend.age, friend.city",
        name=name
    )
    for record in result:
        print(f"Friend: {record['friend.name']}, Age: {record['friend.age']}, City: {record['friend.city']}")

with driver.session() as session:
    session.execute_read(get_friends, name="Alice")

### Change a Relationship

In [21]:
def change_relationship(tx, name1: str, name2: str, old_rel_type: str, new_rel_type: str):
    # Remove the existing relationship
    remove_relationship(tx, name1, name2, old_rel_type)
    # Add the new relationship
    add_relationship(tx, name1, name2, new_rel_type)
    
# Example usage
with driver.session() as session:
    session.execute_write(change_relationship, name1="Alice", name2="Bob", old_rel_type="FRIEND_WITH", new_rel_type="BEST_FRIEND_WITH")

Removed FRIEND_WITH relationship: Alice -> Bob
Added BEST_FRIEND_WITH relationship: Alice -> Bob


### Delete a Node

In [22]:
def delete_person(tx, name: str):
    tx.run("MATCH (p:Person {name: $name}) DETACH DELETE p", name=name)
    print(f"Deleted Person: {name}")

with driver.session() as session:
    session.execute_write(delete_person, name="Bob")

Deleted Person: Bob


### Close the Connection

In [23]:
driver.close()
print("Connection closed.")

Connection closed.
